In [1]:
#Import required libraries for data loading, cleaning and pre processing.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Load the dataset
df = pd.read_csv('train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50



**The dataset consists of 5847 rows and 14 columns**

In [3]:
#Descriptive Statistics
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB
None
        Unnamed: 0         Year  Kilometers_Driven      

## **a) Handling Missing Values**

In [4]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


1. Mileage, Engine, Power, Seats: These columns have only a small portion of missing values relative to the dataset size. Since they are numerical, imputing with the median is suitable as it reduces the influence of outliers.

2. New_Price: With more than 86% of its data missing, it’s best to drop this column, as filling in values would likely be uninformative.

In [5]:
# Extract numeric values and convert the columns
df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.\d+|\d+)').astype(float)  # Remove non-numeric parts and convert to float
df['Engine'] = df['Engine'].str.extract(r'(\d+)').astype(float)
df['Power'] = df['Power'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [6]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Impute median for Mileage, Engine, Power, Seats
df['Mileage'].fillna(df['Mileage'].median(), inplace=True)
df['Engine'].fillna(df['Engine'].median(), inplace=True)
df['Power'].fillna(df['Power'].median(), inplace=True)
df['Seats'].fillna(df['Seats'].median(), inplace=True)

In [7]:
# Drop New_Price due to a high number of missing values
df.drop(columns=['New_Price'], inplace=True)

In [8]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


**We can see that there are no missing values after being imputed.**

## **Step b: The units have already been removed while extracting the numeric columns**



## **C) One hot encoding categorical features**

1. Fuel_Type and Transmission were converted into numerical one-hot encoded columns.

In [9]:
#One-hot encoding categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True


## **d) Creating a New Feature:**
A new column Car_Age was added, representing the car's age (2024 - Year).

In [10]:
#Creating a new feature 'Car_Age'
df['Car_Age'] = 2024 - df['Year']
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True,9
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True,13
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,12
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False,11
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True,11


## **e) SQL-Like Operations:**

In [16]:
# Select specific columns
selected_df = df[['Name', 'Location', 'Price', 'Kilometers_Driven', 'Car_Age', 'Owner_Type']]
selected_df.head()

,Name,Location,Price,Kilometers_Driven,Car_Age,Owner_Type
0,Hyundai Creta 1.6 CRDi SX Option,Pune,12.50,41000,9,First
1,Honda Jazz V,Chennai,4.50,46000,13,First
2,Maruti Ertiga VDI,Chennai,6.00,87000,12,First
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,17.74,40670,11,Second
4,Nissan Micra Diesel XV,Jaipur,3.50,86999,11,First


In [18]:
# Filter rows where Price is greater than 5 and Kilometers driven is greater than 100000
filtered_data = selected_df[(selected_df['Price'] > 5) & (selected_df['Kilometers_Driven'] > 100000)]
filtered_data.head()


,Name,Location,Price,Kilometers_Driven,Car_Age,Owner_Type
13,Mitsubishi Pajero Sport 4X4,Delhi,15.00,110000,10,First
56,Nissan X-Trail SLX AT,Hyderabad,7.75,121812,14,First
75,Toyota Innova 2.5 VX 7 STR BSIV,Hyderabad,11.75,104000,12,First
86,Toyota Fortuner 4x2 Manual,Delhi,16.50,107000,10,First
113,BMW 5 Series 2010-2013 525i,Mumbai,7.00,128000,15,First


In [20]:
# Rename columns (e.g., rename 'Location' to 'Place' and Price to Cost) using groupby
renamed_df = filtered_data.rename(columns={'Location': 'Place', 'Price': 'Cost'})
renamed_df.head()

,Name,Place,Cost,Kilometers_Driven,Car_Age,Owner_Type
13,Mitsubishi Pajero Sport 4X4,Delhi,15.00,110000,10,First
56,Nissan X-Trail SLX AT,Hyderabad,7.75,121812,14,First
75,Toyota Innova 2.5 VX 7 STR BSIV,Hyderabad,11.75,104000,12,First
86,Toyota Fortuner 4x2 Manual,Delhi,16.50,107000,10,First
113,BMW 5 Series 2010-2013 525i,Mumbai,7.00,128000,15,First


In [23]:
# Arrange/sort by 'Age of Car' in ascending order
arranged_data = renamed_df.sort_values(by='Car_Age', ascending=True)
arranged_data.head()

,Name,Place,Cost,Kilometers_Driven,Car_Age,Owner_Type
2259,BMW X5 xDrive 30d M Sport,Chennai,65.00,6500000,7,First
1691,Maruti Ertiga SHVS VDI,Pune,8.00,121000,7,First
4050,Hyundai Creta 1.6 SX Option Diesel,Coimbatore,11.45,101065,7,First
5391,Honda Amaze VX i-DTEC,Hyderabad,6.30,128000,7,First
469,Mercedes-Benz GLS 350d 4MATIC,Pune,56.00,129000,7,First


In [26]:
# Mutate: Add a new feature "Mileage_per_CC"
df['Mileage_per_CC'] = df['Kilometers_Driven'] / df['Engine']
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age,Mileage_per_CC
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True,9,25.916561
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True,13,38.365304
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,12,69.711538
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False,11,20.665650
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True,11,59.547570


In [21]:
# Summarize: Group by 'Power' and calculate average 'Mileage'
summary_df = df.groupby('Power')['Mileage'].mean().reset_index()
summary_df.head()

,Power,Mileage
0,34.20,16.800000
1,35.00,18.164706
2,35.50,12.800000
3,37.00,16.100000
4,37.48,23.400000
